##**Flow Rate Profile Analysis**
Feb 2022 - Lane Breshears, modified from .py code Alex Day

Use with any microfluidic chip, place the black line over the channel to analyze flow across the channel. Specify number of channels and channel gaps.


## Setup

**Link CoLab code with your Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# https://drive.google.com/drive/folders/1FWndAYiDDbhP_JWq7UcodWYYF2mCsjqb?usp=sharing

Mounted at /content/drive


**Link CoLab code with Google Sheets**

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

**Open the active folder with videos**

In [3]:
# Change the below to open up the folder that has the videos in it
!ls '/content/drive/MyDrive/PFAS_Project/videos/chipdip_prototype'

 20221128_115608.mp4  'test bounds.png'


**Import Libraries**

In [4]:
from tensorflow.python.util.tf_export import get_canonical_name_for_symbol
import numpy as np
import cv2
import os
import csv
import matplotlib.pyplot as plt
import sys
from google.colab.patches import cv2_imshow #to see images, can be deleted later
import pandas as pd

import gspread_dataframe as gd
import gspread as gs

# Analyze videos

In [26]:
def createSpreadsheet(videoName):
	'''
	videoName = the name of the video being analyzed will be 
	used to create the worksheet
	'''
	gc = gspread.authorize(creds)
	sh = gc.create(str(videoName))
	print(videoName)
	return sh

def defineLane(start, end):
	'''
	Auxilliary function used to correctly determine which pixels to look at in the lane, regardless of whether the pixel 
	numbers go from high to low or low to high
	'''
	points = []
	temp = end - start
	if temp < 0:
		for i in range(end, start):
			points.append(i)
	else:
		for i in range(start, end):
			points.append(i)
	return points

def flowAnalysis(videoName, location, start, end, channel, axes, threshold=13):
	'''
	Given a video file, return a list of the flow profile of the moving front. Each index will represent a frame of the 
	original video file, and its value will represent how many pixels the wetting front has flowed through

	Parameters:
	"videoName" = the file name of the video file
	"Location" = if the axis is horizontal, then location is the pixel row containing the desired lane, while if the
				 axis vertical, then location is the pixel column contained the desired lane
	"start" = the starting pixel in the row/column (dictated by "Location" parameter) of the desired lane
	"end" = the ending pixel in the row/column (dictated by the "Location" parameter) of the desired lane
	"axes" = determines whether the desired flow lane is horizontal or vertical in the video file (MUST BE EITHER "H" OR "V")
	"channel" = the channel number on the chip
	'''
	
	# NOTE: if you're not seeing any movement in your data, change this threshold value! It represents the brightness change
	# in a pixel within the flow lane that is considered to be high enough to conclude that the moving front has moved to
	# that location. 
	threshold = threshold

	possibleAxes = ['H', 'V']
	if axes not in possibleAxes:
		print('Axes parameter not allowed, must be either H or V')
		sys.exit()
	laneLength = defineLane(start, end)

	flowProfile = []
	video = cv2.VideoCapture(videoName)
	print('Video name: {}'.format(videoName))  #4th thing to happen 
	print('Video length (frames): {}'.format(video.get(cv2.CAP_PROP_FRAME_COUNT))) #5th thing to happen 
	success = True
	imageNumber = 1
	while success:
		# print('Image Number: {}'.format(imageNumber))
		success, image = video.read()
		if success:
			gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
			if imageNumber == 1:
				initialLane = []
				for i in laneLength:
					triplet = []
					for j in range(-1, 2):
						if axes == 'H':
							triplet.append(gray[location + j, i])
						elif axes == 'V':
							triplet.append(gray[i, location + j])
					initialLane.append(np.average(triplet))

			currentLane = []
			differences = 0
			for i in laneLength:
				triplet = []
				for j in range(-1, 2):
					if axes == 'H':
							triplet.append(gray[location + j, i])
					elif axes == 'V':
						triplet.append(gray[i, location + j])
				currentLane.append(np.average(triplet))

			for i in range(0, len(laneLength)):
				difference = initialLane[i] - currentLane[i]
				if difference >= threshold:
					differences += 1

			flowProfile.append(differences)
			imageNumber += 1


	
	print('Maximum flow (pixels): {}'.format(np.amax(flowProfile)))
	videoFPS = video.get(cv2.CAP_PROP_FPS)
	print('VIDEO FPS IS:')
	print(videoFPS)
	videoDuration = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) / videoFPS
	print('Total Velocity: {:2f}'.format(float(np.amax(flowProfile) / videoDuration)))
	print('------------------')

	plt.figure()
	plt.suptitle('Flow Profile (Pixels vs. Frame Number)')
	plt.plot(flowProfile)
	plt.show()
	return flowProfile

def exampleBounds(videoName, location, start, end, axes='H'):
	'''
	Given a video file, the function will save a grayscale image depicting where the analysis algorithm would consider
	the lane to be (by using a solid black line over the first frame of the video)

	Parameters:
	"videoName" = the file name of the video file
	"Location" = if the axis is horizontal, then location is the pixel row containing the desired lane, while if the
				 axis vertical, then location is the pixel column contained the desired lane
	"start" = the starting pixel in the row/column (dictated by "Location" parameter) of the desired lane
	"end" = the ending pixel in the row/column (dictated by the "Location" parameter) of the desired lane
	"axes" = determines whether the desired flow lane is horizontal or vertical in the video file (MUST BE EITHER "H" OR "V")
	'''
	possibleAxes = ['H', 'V']
	if axes not in possibleAxes:
		print('Axes parameter not allowed, must be either H or V')
		sys.exit()
	laneLength = defineLane(start, end)  #uses auxilary function defineLane
	video = cv2.VideoCapture(videoName)  #reads video file from path, still a video file I believe
	#cv2_imshow(video) #added for testing, didn't work with this line
	print(videoName) #2nd thing to happen
	success = True
	success, image = video.read() #or this may be the crux of the cookie actually I think not
	cv2_imshow(image, 300)  #added for testing
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # converts image from one color space to another "image" to gray scale image named "gray"
	for i in laneLength:		#creates the gray lines using iteravite process
		if axes == 'H':
			gray[location, i] = 0 #changes pixel color at specified points 0: black 255: white
		elif axes == 'V':
			gray[i, location] = 0 #changes pixel color at specified points 0: black 255: white
	cv2.imwrite('test bounds.png', gray)  #saves image to specified file 
	cv2_imshow(gray) #shows the image below. 3rd thing to happen


if __name__ == '__main__':

	#EDITABLE FOR DATA COLLECTION 
	os.chdir(os.path.join(r'/content/drive/MyDrive/PFAS_Project/videos/chipdip_prototype'))  #C:\Users\owner\Downloads'))


	videoName = '20221128_115608.mp4'
	start = int(2900) #100 vertical positioning)
	end = int(2100) #510 vertical positioning)
	channel_gap = int(145) #130 - cellulose jump from channel to chnl in px distance
	location1 = int(845) #Row/channel initial position for horizontal chip/Verticla chip would be column (moves horizontally)
	numberChannels = 4 #5 for cellulose - 4 for covid
	axes = 'V' #H - horizontal, V - vertical
	#Save the data to google sheets
	gc = gspread.authorize(creds)
	sh = gc.create(str(videoName))
	
	#END OF EDITABLE
	

	for i in list(range(0,numberChannels)):   #code "starts" here
		channel = i + 1
		if axes == 'H':
			channel_location = location1 - channel_gap*(i) #assuming 1st channel is on bottom of chip
		elif axes == 'V':
			channel_location = location1 + channel_gap*(i) #assuming 1st channel is on far left of chip
		else:
			print('Check which channel you are starting on')
	 
		print(channel_location)  #1st thing to happen
		exampleBounds(videoName, channel_location, start, end, axes) 
		flowProfile_list = flowAnalysis(videoName, channel_location, start, end, i, axes, threshold=9)
		df_flowProfile = pd.DataFrame(flowProfile_list)
		
		ws = sh.add_worksheet(title=str(channel), rows="100", cols="20")
		ws = gc.open(str(videoName)).worksheet(str(channel))
		gd.set_with_dataframe(ws, df_flowProfile)

	ws = sh.get_worksheet(0)
	a=range(len(df_flowProfile)+1)
	array = np.array([a]).T
	# Write the frame col to the worksheet starting from the A1 cell
	ws.update('A1', array.tolist())


845
20221128_115608.mp4


TypeError: ignored

Check your data

In [ ]:
'''
#Click to go directly to the data collection spreadsheet of the video analyzed - use latest version and make sure it has all the data
#Each sheet is a channel

Go to https://sheets.google.com to see your new spreadsheet
'''

'\n#Click to go directly to the data collection spreadsheet of the video analyzed - use latest version and make sure it has all the data\n#Each sheet is a channel\n\nGo to https://sheets.google.com to see your new spreadsheet\n'